<a href="https://colab.research.google.com/github/OnesimusMMC/GWG_Scholar_Selection/blob/main/Scholar_Selection_Agent_V1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import google.ai.generativelanguage
import google.generativeai as genai
import os
from google.colab import files
from google.colab import auth
from google.auth import default
import gspread
from datetime import datetime
import time
from tenacity import retry, stop_after_attempt, wait_fixed
import re  # Import the regular expression library
import numpy as np # Import numpy for nan/inf handling
from google.colab import userdata
import gspread.exceptions # Import specific gspread exceptions

print(f"google-generativeai version: {google.generativeai.__version__}")
print(f"google-ai-generativelanguage version: {google.ai.generativelanguage.__version__}")

# --- 1. Install Libraries (if not already installed in Colab) ---
# !pip install pandas google-generativeai gspread google-auth-httplib2 tenacity numpy

# --- 2. Authenticate Google Sheets API ---
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("Google Sheets API Authentication Successful.")
except Exception as e:
    print(f"Error during Google Sheets Authentication: {e}")
    exit()

# --- 3. Set up Gemini API Key ---
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    if not GEMINI_API_KEY:
        raise ValueError("GEMINI_API_KEY not found in Colab secrets.")
    genai.configure(api_key=GEMINI_API_KEY)
    # Using a specific model known for reliability, adjust if needed
    model = genai.GenerativeModel('gemini-1.5-flash-latest') # Changed model name for potential compatibility/updates
    print("Gemini API Configured Successfully.")
except Exception as e:
    print(f"Error setting up Gemini API: {e}")
    exit()

# --- 4. Prepare Excel Sheet (File Upload) ---
print("Please upload the Excel file containing applicant data.")
try:
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded. Exiting.")
        exit()
    EXCEL_FILE_PATH = next(iter(uploaded))
    print(f"File '{EXCEL_FILE_PATH}' uploaded successfully.")
except Exception as e:
    print(f"Error during file upload: {e}")
    exit()


current_date = datetime.now().strftime("%Y%m%d_%H%M%S") # Added time for uniqueness
OUTPUT_EXCEL_PATH = f"applicant_scores_{current_date}.xlsx"
base_name = os.path.splitext(EXCEL_FILE_PATH)[0]
GOOGLE_SHEET_NAME = f"{base_name}_{current_date}_evaluated"

# --- 5. Load Data from Excel ---
try:
    df = pd.read_excel(EXCEL_FILE_PATH)
    print(f"Successfully loaded data from '{EXCEL_FILE_PATH}'. Found {len(df)} applicants.")
    # Basic validation: Check if DataFrame is empty
    if df.empty:
        print("Warning: The uploaded Excel file is empty.")
    # Optional: Add checks for expected columns if known
    # expected_cols = ['Name', 'Email', 'Application Text'] # Example
    # if not all(col in df.columns for col in expected_cols):
    #    print(f"Warning: Expected columns ({expected_cols}) not found in the Excel file.")

except FileNotFoundError:
    print(f"Error: Excel file not found at {EXCEL_FILE_PATH}")
    exit()
except Exception as e:
    print(f"Error loading data from Excel file: {e}")
    exit()


# --- Define Evaluation Criteria and Scoring Guidelines ---
evaluation_criteria = """
**Use Case:** To create a fair and transparent process for evaluating applicants based on their career stage, goals, and alignment with the organization's mission.

**Key Levels:**
Level A: Career changers and upskillers.
Level B: Entry-level candidates and college students.
Level C: Career professionals seeking certifications but lacking a clear path.
Level D: Individuals submitting AI-generated applications with limited personalization.

**Point-Based Evaluation System:**
Each applicant is scored out of 50 points across four criteria:

**Criteria** | **Max Points** | **Scoring Guidelines**
---|---|---
Grade like a highly rated professor
Clarity of Career Goals | 15 | Clear and specific goals (13–15); Somewhat clear (10–12); Unclear or vague (0–9).
Relevant Experience | 10 | Strong transferable skills (8–10); Some relevant experience (5–7); Minimal (0–4).
Effort Demonstrated | 15 | Evidence of upskilling or proactive learning (13–15); Moderate effort (5–12); None (0–4).
Alignment with Program | 10 | Goals align with mentorship mission (8–10); Partial alignment (3–7); Misaligned (0–2).

**Score Range and Decision:**
40–50: Fully Accepted (Priority Placement).
30–39: Accepted (Standard Placement).
20–29: Waitlisted (Consider for Mentorship).
0-19: Rejected (Not suitable for program at this time). # Added Rejected category for clarity

**AI Detection Penalties:**
- If 71-100% sure AI-generated: Reduce Clarity by 5, Effort by 5, Experience by 3, Alignment by 3.
- If 60-70% sure AI-generated: Reduce Clarity by 4, Effort by 4, Experience by 2, Alignment by 2.
-If 50-59% sure AI-generated: Reduce Clarity by 3, Effort by 3, Experience by 1, Alignment by 1.
- If less than 50% sure AI-generated: No penalty.
"""

@retry(stop=stop_after_attempt(3), wait=wait_fixed(20)) # Retry 3 times, wait 20s between attempts
def evaluate_applicant(applicant_data):
    """Evaluates a single applicant using the Gemini API with retry logic."""
    prompt = f"""{evaluation_criteria}

**Evaluate the following applicant based on the criteria above:**

**Applicant Information:**
{applicant_data}

**Instructions:**
1. Provide a score (0-Max Points) for each of the four criteria.
2. Calculate the total score out of 50.
3. Determine the final decision based on the total score (Fully Accepted, Accepted, Waitlisted, Rejected).
4. Indicate if you detected potential AI generation (and the certainty level if applicable) BEFORE applying penalties.
5. Apply penalties ONLY IF AI generation is detected based on the rules in the criteria. State the adjusted scores and the final decision AFTER penalties.
6. Show the reason behind the scores in the console
**Required Response Format:**
AI Detection Certainty: [e.g., None, 65%, 80%]
Clarity of Career Goals (Initial): [Score]/15
Relevant Experience (Initial): [Score]/10
Effort Demonstrated (Initial): [Score]/15
Alignment with Program (Initial): [Score]/10
Total Score (Initial): [Total Score]/50
--- [Apply Penalties If Applicable] ---
Clarity of Career Goals (Adjusted): [Score]/15
Relevant Experience (Adjusted): [Score]/10
Effort Demonstrated (Adjusted): [Score]/15
Alignment with Program (Adjusted): [Score]/10
Total Score (Adjusted): [Total Score]/50
Final Decision: [Fully Accepted | Accepted | Waitlisted | Rejected]
"""
    try:
        # It's often good to specify safety settings, especially when dealing with user-generated text
        safety_settings = [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]
        response = model.generate_content(prompt, safety_settings=safety_settings)
        # Basic check if response has content
        if response.parts:
             return response.text
        else:
             # Handle cases where the response might be blocked or empty
             print(f"Warning: Received empty or blocked response from Gemini. Finish reason: {response.prompt_feedback.block_reason if response.prompt_feedback else 'N/A'}")
             return "Error: Gemini response was empty or blocked."
    except Exception as e:
        # Catching potential API errors more specifically can be helpful
        print(f"Error during Gemini API call (will retry if possible): {e}")
        # Check for specific API-related errors if the library provides them
        # Example: if isinstance(e, google.api_core.exceptions.ResourceExhausted): print("Rate limit likely exceeded.")
        raise  # Re-raise the exception for tenacity to handle retry


# --- MODIFIED: parse_gemini_response ---
def parse_gemini_response(response_text):
    """
    Parses the Gemini API response to extract scores (initial and adjusted)
    and decision using regular expressions. Returns final adjusted scores.
    """
    # Initialize with None for robustness
    scores = {
        "Clarity of Career Goals": None,
        "Relevant Experience": None,
        "Effort Demonstrated": None,
        "Alignment with Program": None,
        "Total Score": None,
    }
    decision = "Error: Parsing Failed" # Default decision

    # Prefer adjusted scores if available, otherwise fallback to initial
    patterns = {
        "Clarity of Career Goals": r"Clarity of Career Goals \(Adjusted\):\s*(\d+)/15",
        "Relevant Experience": r"Relevant Experience \(Adjusted\):\s*(\d+)/10",
        "Effort Demonstrated": r"Effort Demonstrated \(Adjusted\):\s*(\d+)/15",
        "Alignment with Program": r"Alignment with Program \(Adjusted\):\s*(\d+)/10",
        "Total Score": r"Total Score \(Adjusted\):\s*(\d+)/50",
        # Fallback to initial scores if adjusted are not found
        "Clarity of Career Goals_Initial": r"Clarity of Career Goals \(Initial\):\s*(\d+)/15",
        "Relevant Experience_Initial": r"Relevant Experience \(Initial\):\s*(\d+)/10",
        "Effort Demonstrated_Initial": r"Effort Demonstrated \(Initial\):\s*(\d+)/15",
        "Alignment with Program_Initial": r"Alignment with Program \(Initial\):\s*(\d+)/10",
        "Total Score_Initial": r"Total Score \(Initial\):\s*(\d+)/50",
        "Decision": r"Final Decision:\s*(.+)",
    }

    try:
        # Parse Decision first
        decision_match = re.search(patterns["Decision"], response_text, re.IGNORECASE | re.DOTALL)
        if decision_match:
            decision = decision_match.group(1).strip()
        else:
            print(f"Warning: Could not parse Final Decision from response:\n------\n{response_text}\n------")
            # Attempt to determine decision based on score if possible later

        # Parse Scores (prioritize Adjusted)
        for key in ["Clarity of Career Goals", "Relevant Experience", "Effort Demonstrated", "Alignment with Program", "Total Score"]:
            adjusted_match = re.search(patterns[key], response_text, re.IGNORECASE)
            if adjusted_match:
                try:
                    scores[key] = int(adjusted_match.group(1))
                except ValueError:
                    print(f"Warning: Could not convert adjusted score to int for {key}.")
            else:
                # If adjusted not found, try initial
                initial_match = re.search(patterns[key + "_Initial"], response_text, re.IGNORECASE)
                if initial_match:
                    try:
                        scores[key] = int(initial_match.group(1))
                        print(f"Info: Using initial score for {key} as adjusted was not found.")
                    except ValueError:
                        print(f"Warning: Could not convert initial score to int for {key}.")
                else:
                     print(f"Warning: Could not parse score (Adjusted or Initial) for {key}.")
                     # score[key] remains None

        # Validate/Recalculate Total Score if needed
        parsed_total = scores["Total Score"]
        individual_scores = [scores["Clarity of Career Goals"], scores["Relevant Experience"], scores["Effort Demonstrated"], scores["Alignment with Program"]]

        if None not in individual_scores:
             calculated_total = sum(individual_scores)
             if parsed_total is None:
                 scores["Total Score"] = calculated_total
                 print(f"Info: Calculated Total Score as {calculated_total} (was missing).")
             elif parsed_total != calculated_total:
                 print(f"Warning: Parsed Total Score ({parsed_total}) does not match calculated sum ({calculated_total}). Using calculated value.")
                 scores["Total Score"] = calculated_total
        elif parsed_total is None:
             print("Warning: Could not parse or calculate Total Score (missing individual scores).")

        # Fallback for decision if parsing failed but score exists
        if decision == "Error: Parsing Failed" and scores["Total Score"] is not None:
            score = scores["Total Score"]
            if 40 <= score <= 50: decision = "Fully Accepted"
            elif 30 <= score <= 39: decision = "Accepted"
            elif 20 <= score <= 29: decision = "Waitlisted"
            elif 0 <= score <= 19: decision = "Rejected"
            else: decision = "Error: Score out of range"
            print(f"Info: Determined decision '{decision}' based on calculated/parsed score.")


    except Exception as e:
        print(f"Error during response parsing: {e}\nResponse Text:\n------\n{response_text}\n------")
        # Return dict with Nones and default error decision
        return {k: None for k in scores}, "Error: Exception during Parsing"

    return scores, decision

# --- 6. Iterate Through Applicants and Evaluate ---
all_scores_list = [] # Use a list of dictionaries
all_decisions = []
raw_responses = []
failed_applicants = []
processed_count = 0
# Consider making rate limit configurable or checking API documentation
# Free tier is often 60 QPM (Queries Per Minute) -> 1 query per second
# Pausing 60s after 15 calls is very conservative, could be faster e.g. 60s after 50 calls
rate_limit_calls = 15 # Number of calls before pausing
sleep_duration = 60 # Seconds to sleep (a bit more than 60s for safety)

start_time = time.time()

print(f"\n--- Starting Applicant Evaluation (Rate Limit: {rate_limit_calls} calls / {sleep_duration}s pause) ---")

for index, row in df.iterrows():
    # Handle potential non-string data in rows before joining
    try:
        applicant_info = "\n".join([f"{col}: {str(row[col])}" for col in df.columns if pd.notna(row[col])])
    except Exception as e:
        print(f"Error formatting applicant info for row {index + 1}: {e}. Skipping applicant.")
        all_scores_list.append({}) # Append empty dict
        all_decisions.append("Error: Formatting Applicant Info")
        raw_responses.append("Error formatting applicant data")
        failed_applicants.append((index, "Error formatting applicant data", str(e)))
        continue # Skip to the next applicant

    print(f"Evaluating applicant {index + 1}/{len(df)}...")

    try:
        gemini_response = evaluate_applicant(applicant_info)
        raw_responses.append(gemini_response) # Store raw response regardless of parsing success

        if "Error: Gemini response was empty or blocked." in gemini_response:
             scores = {k: None for k in ["Clarity of Career Goals", "Relevant Experience", "Effort Demonstrated", "Alignment with Program", "Total Score"]}
             decision = "Error: Gemini Response Blocked/Empty"
             print(f"Failed processing applicant {index + 1} due to blocked/empty response.")
             failed_applicants.append((index, applicant_info, gemini_response))
        elif gemini_response.startswith("Error:"): # Catch other potential errors returned as strings
             scores = {k: None for k in ["Clarity of Career Goals", "Relevant Experience", "Effort Demonstrated", "Alignment with Program", "Total Score"]}
             decision = gemini_response # Store the specific error message
             print(f"Failed processing applicant {index + 1} due to Gemini error string: {decision}")
             failed_applicants.append((index, applicant_info, gemini_response))
        else:
            scores, decision = parse_gemini_response(gemini_response) # Parse the valid response

        all_scores_list.append(scores)
        all_decisions.append(decision)

        # Optionally print parsed results for verification
        # print(f"  Parsed Scores: {scores}")
        # print(f"  Decision: {decision}")

    except Exception as e:
        # This catches errors from evaluate_applicant (after retries) or unexpected errors in the loop
        print(f"Failed to process applicant {index + 1} after retries or due to unexpected error: {e}")
        # Append placeholders
        all_scores_list.append({k: None for k in ["Clarity of Career Goals", "Relevant Experience", "Effort Demonstrated", "Alignment with Program", "Total Score"]})
        all_decisions.append("Error: Processing Failed")
        raw_responses.append(f"Error during processing loop: {e}")
        failed_applicants.append((index, applicant_info, f"API call/Loop failed: {e}"))

    processed_count += 1
    # Rate Limiting Logic
    if processed_count % rate_limit_calls == 0 and index < len(df) - 1:
        print(f"\n--- Reached rate limit ({rate_limit_calls} calls). Pausing for {sleep_duration} seconds... ---\n")
        time.sleep(sleep_duration)
        print("--- Resuming evaluation... ---")


end_time = time.time()
print(f"\n--- Evaluation Complete ---")
print(f"Total time taken: {end_time - start_time:.2f} seconds")
print(f"Processed {processed_count} applicants.")
success_count = len([d for d in all_decisions if not d.startswith("Error:")])
print(f"Successfully evaluated: {success_count}")
print(f"Failed/Errored evaluations: {processed_count - success_count}")

# --- 7. Store and Output Results ---

# Create DataFrame from the list of score dictionaries
scores_df = pd.DataFrame(all_scores_list)

# Define expected columns to ensure they exist, even if all parsing failed
expected_score_cols = ["Clarity of Career Goals", "Relevant Experience", "Effort Demonstrated", "Alignment with Program", "Total Score"]
for col in expected_score_cols:
    if col not in scores_df.columns:
        scores_df[col] = None # Add missing score columns initialized with None

# Ensure correct column order and add Decision
scores_df = scores_df[expected_score_cols]
scores_df['Decision'] = all_decisions

# Concatenate original data with scores and decisions
# Reset index on both DataFrames before concat to ensure alignment
df_reset = df.reset_index(drop=True)
scores_df_reset = scores_df.reset_index(drop=True)
final_df = pd.concat([df_reset, scores_df_reset], axis=1)

# --- Output to Google Sheet ---
try:
    print("\nPreparing data for Google Sheets...")
    # Replace NaN/NaT with empty strings, which are JSON compliant and show as blank cells
    # Convert ALL data to strings before sending for maximum compatibility with gspread/JSON
    final_df_str = final_df.fillna('').astype(str)

    header = final_df_str.columns.tolist()
    data = final_df_str.values.tolist()

    print(f"Creating Google Sheet named: {GOOGLE_SHEET_NAME}")
    spreadsheet = gc.create(GOOGLE_SHEET_NAME)
    worksheet = spreadsheet.sheet1 # Get the first sheet

    print("Updating Google Sheet...")
    # Use 'USER_ENTERED' to let Sheets interpret types if possible, though we send strings
    worksheet.update([header] + data, value_input_option='USER_ENTERED')

    # Share the sheet (optional, replace with your email or make public)
    # spreadsheet.share('your_email@example.com', perm_type='user', role='writer')
    # print(f"Sharing Sheet with your_email@example.com")

    print(f"✅ Evaluation results successfully saved to Google Sheet:")
    print(f"   ➡️ {spreadsheet.url}")

except gspread.exceptions.APIError as e:
     # Provide more specific feedback for API errors
     print(f"❌ Google Sheets API Error: {e}")
     print("   This might be due to permissions, quotas, or invalid data format.")
     print("   Check the full error message above and Google Cloud Console for details.")
except Exception as e:
    print(f"❌ Error saving results to Google Sheet: {e}")
    print("   Ensure Google Sheets API is enabled in your Google Cloud project.")

# --- Output to Downloadable Excel Sheet ---
try:
    print(f"\nSaving results to Excel file: {OUTPUT_EXCEL_PATH}...")
    # Fill NaN with empty strings for cleaner Excel output as well
    final_df_excel = final_df.fillna('')
    final_df_excel.to_excel(OUTPUT_EXCEL_PATH, index=False, engine='openpyxl') # Specify engine if needed
    files.download(OUTPUT_EXCEL_PATH)
    print(f"✅ Evaluation results saved to {OUTPUT_EXCEL_PATH} and download initiated.")
except Exception as e:
    print(f"❌ Error saving results to Excel: {e}")

# --- Optional: Save failed responses ---
if failed_applicants:
    print(f"\nSaving details of {len(failed_applicants)} failed evaluations to 'failed_applicants.csv'...")
    try:
        failed_df = pd.DataFrame(failed_applicants, columns=["Original Index", "Applicant Info Snippet", "Error/Raw Response"])
        # Truncate long applicant info/response for CSV readability
        failed_df["Applicant Info Snippet"] = failed_df["Applicant Info Snippet"].str.slice(0, 500)
        failed_df["Error/Raw Response"] = failed_df["Error/Raw Response"].str.slice(0, 1000)
        failed_df.to_csv("failed_applicants.csv", index=False)
        print("✅ Saved failed applicant evaluations to 'failed_applicants.csv'")
        files.download("failed_applicants.csv") # Also download the failed log
    except Exception as e:
        print(f"❌ Error saving failed applicant log: {e}")

print("\n--- Script Finished ---")

google-generativeai version: 0.8.4
google-ai-generativelanguage version: 0.6.15
Google Sheets API Authentication Successful.
Gemini API Configured Successfully.
Please upload the Excel file containing applicant data.


Saving 150_Grow_with_Google_Application_Spring_2025_1743351004 copy.xlsx to 150_Grow_with_Google_Application_Spring_2025_1743351004 copy.xlsx
File '150_Grow_with_Google_Application_Spring_2025_1743351004 copy.xlsx' uploaded successfully.
Successfully loaded data from '150_Grow_with_Google_Application_Spring_2025_1743351004 copy.xlsx'. Found 150 applicants.

--- Starting Applicant Evaluation (Rate Limit: 15 calls / 60s pause) ---
Evaluating applicant 1/150...
Evaluating applicant 2/150...
Evaluating applicant 3/150...
Evaluating applicant 4/150...
Evaluating applicant 5/150...
Evaluating applicant 6/150...
Evaluating applicant 7/150...
Evaluating applicant 8/150...
Evaluating applicant 9/150...
Evaluating applicant 10/150...
Evaluating applicant 11/150...
Evaluating applicant 12/150...
Evaluating applicant 13/150...
Evaluating applicant 14/150...
Evaluating applicant 15/150...

--- Reached rate limit (15 calls). Pausing for 60 seconds... ---

--- Resuming evaluation... ---
Evaluating ap

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Evaluation results saved to applicant_scores_20250413_223210.xlsx and download initiated.

--- Script Finished ---
